<h1 align='center'><font size =10 color='gold'>Segmentation and Clustering Neighborhood in Toronto --3</font></h1>
<h3>By Gaurav Bhardwaj</h3>

Importing Modules required for this `Notebook`: 

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Visualizing the collected Data

In [2]:
link = "https://raw.githubusercontent.com/sample1234-rgb/Coursera_Capstone/master/New_dataFrame.csv"
neighborhood = pd.read_csv(link,index_col=0)

neighborhood.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge ,Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [3]:
print("The data set contains {} Boroughs & {} neighbors.".format(len(neighborhood['Borough'].unique()),neighborhood.shape[0]))

The data set contains 11 Boroughs & 103 neighbors.


Collecting Latitude,Longitude: 

In [4]:
address = 'Toronto,Canada'

geolocator = Nominatim(user_agent="Cnda_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Map:

In [5]:
# create map of New York using latitude and longitude values
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbor in zip(neighborhood['Latitude'], neighborhood['Longitude'], neighborhood['Borough'], neighborhood['Neighbourhood']):
    label = '{}, {}'.format(neighbor, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#67f018',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [6]:
CLIENT_ID = '0CUQM4H3VK42LTZQWFLY10FZV5WQGILPY0JW53R4NOUPPDJP' # your Foursquare ID
CLIENT_SECRET = 'SYY513OP1OTKMWBPRRSJBTCQVV0GIVWCAKIJHYX2ZDNPWM5E' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:\n','CLIENT_ID:' , CLIENT_ID,'\nCLIENT_SECRET:' , CLIENT_SECRET)

Your credentails:
 CLIENT_ID: 0CUQM4H3VK42LTZQWFLY10FZV5WQGILPY0JW53R4NOUPPDJP 
CLIENT_SECRET: SYY513OP1OTKMWBPRRSJBTCQVV0GIVWCAKIJHYX2ZDNPWM5E


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, lat, 
            lng, v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Venues Around Toronto:

In [8]:
toronto_venues = getNearbyVenues(names=neighborhood['Neighbourhood'],latitudes=neighborhood['Latitude'],longitudes=neighborhood['Longitude'])

Rouge ,Malvern
Highland Creek ,Rouge Hill ,Port Union
Guildwood ,Morningside ,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park ,Ionview ,Kennedy Park
Clairlea ,Golden Mile ,Oakridge
Cliffcrest ,Cliffside ,Scarborough Village West
Birch Cliff ,Cliffside West
Dorset Park ,Scarborough Town Centre ,Wexford Heights
Maryvale ,Wexford
Agincourt
Clarks Corners ,Sullivan ,Tam O'Shanter
Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview ,Henry Farm ,Oriole
Bayview Village
Silver Hills ,York Mills
Newtonbrook ,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park ,Don Mills South
Bathurst Manor ,Downsview North ,Wilson Heights
Northwood Park ,York University
CFB Toronto ,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens ,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West ,

In [9]:
print(toronto_venues.shape)
toronto_venues.head()

(1346, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge ,Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge ,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
4,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar


In [10]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide ,King ,Richmond",30,30,30,30,30,30
Agincourt,5,5,5,5,5,5
"Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East",3,3,3,3,3,3
"Albion Gardens ,Beaumond Heights ,Humbergate ,Jamestown ,Mount Olive ,Silverstone ,South Steeles ,Thistletown",9,9,9,9,9,9
"Alderwood ,Long Branch",10,10,10,10,10,10
"Bathurst Manor ,Downsview North ,Wilson Heights",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park ,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,30,30,30,30,30,30


In [11]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


In [12]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge ,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide ,King ,Richmond----
                venue  freq
0                Café  0.10
1         Coffee Shop  0.10
2  Seafood Restaurant  0.03
3    Sushi Restaurant  0.03
4          Steakhouse  0.03


----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Clothing Store   0.2
2               Skating Rink   0.2
3             Breakfast Spot   0.2
4                     Lounge   0.2


----Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East----
                       venue  freq
0                       Park  0.33
1                 Playground  0.33
2           Sculpture Garden  0.33
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Albion Gardens ,Beaumond Heights ,Humbergate ,Jamestown ,Mount Olive ,Silverstone ,South Steeles ,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1            Beer Store  0.11
2   Fried Chicken Joint  0.11
3   Japanese Restaurant  0.11
4  Fast Food Restauran

4                     Lounge   0.0


----Humewood-Cedarvale----
          venue  freq
0    Playground   0.2
1       Dog Run   0.2
2         Field   0.2
3  Hockey Arena   0.2
4         Trail   0.2


----Kingsview Village ,Martin Grove Gardens ,Richview Gardens ,St. Phillips----
                venue  freq
0         Pizza Place  0.25
1      Sandwich Place  0.25
2            Bus Line  0.25
3   Mobile Phone Shop  0.25
4  Mexican Restaurant  0.00


----Kingsway Park South West ,Mimico NW ,The Queensway West ,Royal York South West ,South of Bloor----
                    venue  freq
0                     Gym  0.08
1  Thrift / Vintage Store  0.08
2    Fast Food Restaurant  0.08
3            Burger Joint  0.08
4          Sandwich Place  0.08


----L'Amoreaux West----
                venue  freq
0  Chinese Restaurant   0.2
1            Pharmacy   0.1
2         Pizza Place   0.1
3      Sandwich Place   0.1
4       Grocery Store   0.1


----Lawrence Heights ,Lawrence Manor----
                    

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide ,King ,Richmond",Coffee Shop,Café,Japanese Restaurant,Steakhouse,Bakery,Speakeasy,Hotel,Smoke Shop,Seafood Restaurant,Restaurant
1,Agincourt,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Latin American Restaurant,Women's Store,Curling Ice,Dog Run,Distribution Center,Discount Store
2,"Agincourt North ,L'Amoreaux East ,Milliken ,St...",Park,Playground,Sculpture Garden,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
3,"Albion Gardens ,Beaumond Heights ,Humbergate ,...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Beer Store,Fast Food Restaurant,Japanese Restaurant,Sandwich Place,Women's Store,Dance Studio
4,"Alderwood ,Long Branch",Pizza Place,Athletics & Sports,Gym,Coffee Shop,Sandwich Place,Dance Studio,Pub,Skating Rink,Pharmacy,College Stadium


In [16]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 3, 3, 3, 3, 3, 3])

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Cluster Labels,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,43.806686,-79.194353,Scarborough,"Rouge ,Malvern",0.0,0.0,Fast Food Restaurant,Print Shop,Women's Store,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",0.0,0.0,Golf Course,Bar,Home Service,College Rec Center,College Stadium,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood ,Morningside ,West Hill",0.0,0.0,Intersection,Breakfast Spot,Rental Car Location,Mexican Restaurant,Bank,Medical Center,Electronics Store,Diner,Dim Sum Restaurant,Curling Ice
3,M1G,43.770992,-79.216917,Scarborough,Woburn,3.0,3.0,Coffee Shop,Korean Restaurant,Soccer Field,Cupcake Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae,0.0,0.0,Athletics & Sports,Caribbean Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Dance Studio,Deli / Bodega


In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1.0],
        fill=True,
        fill_color=rainbow[cluster-1.0],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

### Cluster-1:

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.806686,0.0,0.0,Fast Food Restaurant,Print Shop,Women's Store,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
1,43.784535,0.0,0.0,Golf Course,Bar,Home Service,College Rec Center,College Stadium,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,43.763573,0.0,0.0,Intersection,Breakfast Spot,Rental Car Location,Mexican Restaurant,Bank,Medical Center,Electronics Store,Diner,Dim Sum Restaurant,Curling Ice
4,43.773136,0.0,0.0,Athletics & Sports,Caribbean Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Dance Studio,Deli / Bodega
5,43.744734,0.0,0.0,Jewelry Store,Playground,Convenience Store,Women's Store,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
7,43.711112,0.0,0.0,Bakery,Bus Line,Park,Intersection,Ice Cream Shop,Bus Station,Metro Station,Curling Ice,Distribution Center,Discount Store
10,43.757410,0.0,0.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Women's Store,Dessert Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store
11,43.750071,0.0,0.0,Auto Garage,Shopping Mall,Breakfast Spot,Sandwich Place,Bakery,Middle Eastern Restaurant,Women's Store,Dog Run,Distribution Center,Discount Store
12,43.794200,0.0,0.0,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Latin American Restaurant,Women's Store,Curling Ice,Dog Run,Distribution Center,Discount Store
13,43.781638,0.0,0.0,Pizza Place,Noodle House,Chinese Restaurant,Italian Restaurant,Bank,Fast Food Restaurant,Thai Restaurant,Fried Chicken Joint,Convenience Store,Pharmacy


Calculating `NaN` Values:

In [25]:
toronto_merged['Cluster Labels'].isnull().count()

103

### Cluster-2:

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,43.789053,1.0,1.0,Park,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
23,43.752758,1.0,1.0,Park,Convenience Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
50,43.679563,1.0,1.0,Park,Playground,Trail,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
74,43.689026,1.0,1.0,Park,Pool,Women's Store,Gay Bar,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
98,43.706876,1.0,1.0,Park,Convenience Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


### Cluster-3:

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,43.75749,2.0,2.0,Cafeteria,Women's Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


### Cluster-4:

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,43.770992,3.0,3.0,Coffee Shop,Korean Restaurant,Soccer Field,Cupcake Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
6,43.727929,3.0,3.0,Coffee Shop,Bus Station,Department Store,Convenience Store,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
8,43.716316,3.0,3.0,American Restaurant,Motel,Women's Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
9,43.692657,3.0,3.0,General Entertainment,Skating Rink,College Stadium,Café,Women's Store,Cupcake Shop,Dog Run,Distribution Center,Discount Store,Diner
15,43.799525,3.0,3.0,Chinese Restaurant,Pharmacy,Bank,Fast Food Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Grocery Store,Breakfast Spot,Dim Sum Restaurant
18,43.778517,3.0,3.0,Coffee Shop,Clothing Store,Bank,Restaurant,Pharmacy,Bakery,Fast Food Restaurant,Liquor Store,Electronics Store,Salon / Barbershop
19,43.786947,3.0,3.0,Japanese Restaurant,Chinese Restaurant,Bank,Café,Women's Store,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store
22,43.770120,3.0,3.0,Ramen Restaurant,Coffee Shop,Café,Sandwich Place,Restaurant,Shopping Mall,Japanese Restaurant,Bubble Tea Shop,Indonesian Restaurant,Fast Food Restaurant
24,43.782736,3.0,3.0,Coffee Shop,Grocery Store,Bank,Pizza Place,Butcher,Pharmacy,Cupcake Shop,Dog Run,Distribution Center,Discount Store
26,43.745906,3.0,3.0,Japanese Restaurant,Caribbean Restaurant,Café,Gym / Fitness Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Women's Store


### Cluster-5:

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,Cluster Labels,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,43.756303,4.0,4.0,Furniture / Home Store,Women's Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


<h1 align= 'center'>Thank You</h1>